In [1]:
import openai
from dotenv import load_dotenv
import os
import json

load_dotenv()
API_KEY = os.getenv("API_KEY")
config = {
    "API_KEY": API_KEY,
    "model": "gpt-3.5-turbo",
    "store": True,
}
openai.api_key = API_KEY

In [2]:
models = openai.fine_tuning.jobs.list()
print(models)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-2lIadOpy8BV2efd6b5sFDRfW', created_at=1741520769, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:andre-personal:q1-epoch-3-batch-8:B99UIgsW', finished_at=1741521120, hyperparameters=Hyperparameters(batch_size=8, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-FwEmbhh2UoZiY6sQeo5uowVJ', result_files=['file-6h6YtFZHVA5QtXWio2Rvtk'], seed=1160947212, status='succeeded', trained_tokens=151824, training_file='file-Rimsx2EtxaWUiQaUM6265g', validation_file='file-LwCVEPCAi1scGNiNQxxNYo', estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=8, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix='Q1-epoch-3-batch-8'), FineTuningJob(id='ftjob-qbxPcbVQV7Ip4yLP66AhoSl4', created_at=174

In [19]:
# Fine-tuned model ID
# Q1
# fine_tuned_model = "ft:gpt-3.5-turbo-0125:andre-personal:q1-epoch-3-batch-8:B99UIgsW"

# Q2
# fine_tuned_model = "ft:gpt-3.5-turbo-0125:andre-personal:q2-epoch-3-batch-8:B9C2KwaN"

# Q3-full
# fine_tuned_model = "ft:gpt-3.5-turbo-0125:andre-personal:q3-epoch-3-batch-8:B9CDYR15"

# Q3-11
fine_tuned_model = "ft:gpt-3.5-turbo-0125:andre-personal:q3-epoch-3-batch-8:B9CDYIQC:ckpt-step-11"

# Q4
# fine_tuned_model = "ft:gpt-3.5-turbo-0125:andre-personal:q4-epoch-3-batch-8:B9CFAmn9"

# Load test dataset
test_file = "./dataset/processed/test_Q3.jsonl"
output_file = "./output/predictions_Q3_11.jsonl"

predictions = []
if os.path.exists(output_file):
    with open(output_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            predictions.append(data)


cou = 0
continuee = len(predictions)

with open(test_file, "r", encoding="utf-8") as f:
    print("Number of lines in test file:", len(f.readlines()))
    f.seek(0)
    for line in f:
        cou += 1
        print("Processing line", cou)
        if cou <= continuee:
            print("Skipping line", cou)
            continue
        data = json.loads(line)

        # Extract messages from test data
        messages = data["messages"]
        idd = data["id"]
        # print(idd, messages)

        # Perform inference
        response = openai.chat.completions.create(
            model=fine_tuned_model,
            messages=messages,
        )

        # Extract predicted score
        predicted_score = float(response.choices[0].message.content.split(" ")[-1])

        # Save result with ID
        result = {"id": data["id"], "predicted_score": predicted_score}
        predictions.append(result)

# Save predictions as JSONL
with open(output_file, "w", encoding="utf-8") as f:
    for entry in predictions:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ Predictions saved in {output_file}")


Number of lines in test file: 23
Processing line 1
Processing line 2
Processing line 3
Processing line 4
Processing line 5
Processing line 6
Processing line 7
Processing line 8
Processing line 9
Processing line 10
Processing line 11
Processing line 12
Processing line 13
Processing line 14
Processing line 15
Processing line 16
Processing line 17
Processing line 18
Processing line 19
Processing line 20
Processing line 21
Processing line 22
Processing line 23
✅ Predictions saved in ./output/predictions_Q3_11.jsonl


In [9]:
predictions = []
with open(output_file, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        predictions.append(data)

print(predictions)

[{'id': 372, 'predicted_score': 5.0}, {'id': 374, 'predicted_score': 2.0}, {'id': 375, 'predicted_score': 5.0}, {'id': 381, 'predicted_score': 1.0}, {'id': 390, 'predicted_score': 4.0}]


In [8]:
with open(output_file, "w", encoding="utf-8") as f:
    for entry in predictions:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"✅ Predictions saved in {output_file}")

✅ Predictions saved in ./output/predictions_Q1_2.jsonl


In [20]:
q1_ans_path = "./output/predictions_Q1.jsonl"
q2_ans_path = "./output/predictions_Q2.jsonl"
q3_ans_path = "./output/predictions_Q3_11.jsonl"
q4_ans_path = "./output/predictions_Q4.jsonl"

out_path = "./output/predictions_all_2.csv"

path_all_ans = [q1_ans_path, q2_ans_path, q3_ans_path, q4_ans_path]

col = ["ID", "score"]
all_ans = []

for path in path_all_ans:
    with open (path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            all_ans.append(data)

all_ans = sorted(all_ans, key=lambda x: x["id"])
print(all_ans)

with open(out_path, "w", encoding="utf-8") as f:
    f.write(",".join(col) + "\n")
    for entry in all_ans:
        f.write(f"{entry['id']},{entry['predicted_score']}\n")


        

[{'id': 362, 'predicted_score': 5.0}, {'id': 363, 'predicted_score': 2.0}, {'id': 364, 'predicted_score': 1.5}, {'id': 365, 'predicted_score': 5.0}, {'id': 366, 'predicted_score': 5.0}, {'id': 367, 'predicted_score': 5.0}, {'id': 368, 'predicted_score': 5.0}, {'id': 369, 'predicted_score': 0.0}, {'id': 370, 'predicted_score': 5.0}, {'id': 371, 'predicted_score': 2.0}, {'id': 372, 'predicted_score': 5.0}, {'id': 373, 'predicted_score': 1.0}, {'id': 374, 'predicted_score': 2.0}, {'id': 375, 'predicted_score': 5.0}, {'id': 376, 'predicted_score': 4.0}, {'id': 377, 'predicted_score': 5.0}, {'id': 378, 'predicted_score': 0.0}, {'id': 379, 'predicted_score': 0.0}, {'id': 380, 'predicted_score': 5.0}, {'id': 381, 'predicted_score': 1.0}, {'id': 382, 'predicted_score': 3.5}, {'id': 383, 'predicted_score': 4.0}, {'id': 384, 'predicted_score': 3.5}, {'id': 385, 'predicted_score': 0.0}, {'id': 386, 'predicted_score': 5.0}, {'id': 387, 'predicted_score': 0.0}, {'id': 388, 'predicted_score': 5.0}, 